Verify the sobol sequence as used in the CosmoGrid

In [1]:
import os, h5py
import numpy as np

from sobol_seq import i4_sobol
from scipy.stats.qmc import Sobol
from chaospy import create_sobol_samples

from icecream import ic

from msfm.utils import cosmogrid
from msfm.utils.input_output import read_yaml

In [2]:
cosmo_params_info = cosmogrid.get_cosmo_params_info("../../data/CosmoGridV1_original_metainfo.h5", "grid")

### verify with respect to the CosmoGrid
Differences in `Ob` are expected due to bugs in concept

In [3]:
# order like the Sobol sequence
labels = ["Om", "s8", "H0", "Ob", "ns", "w0"]
cosmo_grid_priors = np.array(
    [
        [0.1, 0.5],
        [0.4, 1.4],
        [64.0, 82.0],
        [0.03, 0.06],
        [0.87, 1.07],
        [-2.0, -0.333],
    ], dtype=np.float32
)

# TODO There's a bug with respect to h!

In [4]:
for i in range(len(cosmo_params_info)):
    cosmo_params = [cosmo_params_info[label][i] for label in labels]
    cosmo_params = np.array(cosmo_params, dtype=np.float32)

    i_sobol = cosmo_params_info["sobol_index"][i]
    sobol_point, _ = i4_sobol(6, i_sobol)
    sobol_params = sobol_point*np.squeeze(np.diff(cosmo_grid_priors)) + cosmo_grid_priors[:,0]
    sobol_params = sobol_params.astype(np.float32)

    if i_sobol == 60611:
        print(cosmo_params[2])
        print(sobol_params[2], "\n")

# in /home/ipa/refreg/data/data_products/CosmoGrid/raw/grid/cosmo_060611/run_0/params.yml:
# 68.8507385254

68.85074
73.063446 



In [5]:
counter = 0
for i in range(10):
# for i in range(len(cosmo_params_info)):
    cosmo_params = [cosmo_params_info[label][i] for label in labels]
    cosmo_params = np.array(cosmo_params, dtype=np.float32)

    i_sobol = cosmo_params_info["sobol_index"][i]

    # sobol_seq
    sobol_point, _ = i4_sobol(6, i_sobol)
    sobol_params = sobol_point*np.squeeze(np.diff(cosmo_grid_priors)) + cosmo_grid_priors[:,0]
    sobol_params = sobol_params.astype(np.float32)

    # chaos_py
    sobol_point_2 = np.squeeze(create_sobol_samples(1, 6, i_sobol))
    sobol_params_2 = sobol_point_2*np.squeeze(np.diff(cosmo_grid_priors)) + cosmo_grid_priors[:,0]
    sobol_params_2 = sobol_params_2.astype(np.float32)
    

    print("\n", i_sobol)
    # print(np.isclose(sobol_params, cosmo_params, rtol=1e-3, atol=1e-5), "\n")

    # print(cosmo_params)
    # print(sobol_params)

    # Ob
    print(cosmo_params[3])
    print(sobol_params[3])
    print(sobol_params_2[3])

    # H0
    # print(cosmo_params[2])
    # print(sobol_params[2], "\n")

    if not np.isclose(cosmo_params[2], sobol_params[2], rtol=1e-3, atol=1e-5):
        counter += 1

print(counter)


 1
0.045
0.044999998
0.044999998

 2
0.0525
0.037499998
0.037499998

 3
0.0375
0.0525
0.0525

 4
0.04875
0.033749998
0.033749998

 6
0.04125
0.041249998
0.041249998

 10
0.046875
0.058125
0.058125

 11
0.031875
0.043125
0.043125

 12
0.058125
0.046875
0.046875

 14
0.035625
0.054375
0.054375

 19
0.0478125
0.049687497
0.049687497
8


### single example

In [6]:
# from https://cosmo-gitlab.phys.ethz.ch/jafluri/cosmogrid_kids1000/-/blob/master/kids1000_analysis/constants.py#L13
priors = np.array([[ 0.1  ,  0.5  ],
                   [ 0.4  ,  1.4  ],
                   [ 0.64 ,  0.82 ],
                   [ 0.03 ,  0.06 ],
                   [ 0.87 ,  1.07 ],
                   [-2.   , -0.333],
          np.log10([1.0e12, 1.0e15]),
                   [-2.   ,  2.   ],
                   [-3.   ,  3.   ]])

# /home/ipa/refreg/data/data_products/CosmoGrid/raw/grid/cosmo_011062/run_0
sobol_seed = 11062

# https://cosmo-gitlab.phys.ethz.ch/jafluri/cosmogrid_kids1000/-/blob/master/kids1000_analysis/input_pipeline.py#L461
sobol_point, _ = i4_sobol(9, sobol_seed)
Om, s8, h, Ob, ns, w0, log10_Mc, nu, a_ia = (sobol_point*np.squeeze(np.diff(priors)) + priors[:,0])

ic(Om)
ic(s8)
ic(h)
ic(Ob)
ic(ns)
ic(w0)
ic(log10_Mc)
ic(nu)
ic(a_ia)

# content of params.yml
# As: 8.122750437129466e-10
# bary_Mc: 2943968138429.53
# bary_nu: 1.037841796875
# H0: 68.5758056641
# O_cdm: 0.336458858089
# O_nu: 0.0013725628089999858
# Ob: 0.045737915039
# Ol: -1.0
# Om: 0.383569335937
# m_nu: 0.02
# ns: 0.990397949219
# pkd_seed: 222536
# s8: 0.618078613281
# sobol_seed: 11062
# w0: -1.073199279785
# wa: 0.0

ic| Om: 0.3835693359375
ic| s8: 0.61807861328125
ic| h: 0.734427490234375
ic| Ob: 0.0376263427734375
ic| ns: 0.99039794921875
ic| w0: -1.0731992797851562
ic| log10_Mc: 12.46893310546875
ic| nu: 1.037841796875
ic| a_ia: 2.3565673828125


2.3565673828125

### comparison to scipy (not necessary)

In [7]:
create_sobol_samples(1, 6, 10)

array([[0.9375],
       [0.0625],
       [0.5625],
       [0.9375],
       [0.3125],
       [0.4375]])

In [8]:
print(Sobol(d=5, scramble=False).fast_forward(10).random())
sobol_point, _ = i4_sobol(5, 10)
print(sobol_point)

[[0.9375 0.0625 0.6875 0.1875 0.3125]]
[0.9375 0.0625 0.5625 0.9375 0.3125]


In [9]:
sobol_point, _ = i4_sobol(5, 10)
print(sobol_point)
sobol_point, _ = i4_sobol(10, 10)
print(sobol_point)


[0.9375 0.0625 0.5625 0.9375 0.3125]
[0.9375 0.0625 0.5625 0.9375 0.3125 0.4375 0.8125 0.6875 0.4375 0.8125]


# config

In [10]:
with h5py.File("../../data/CosmoGridV1_original_metainfo.h5") as f:
    meta_info = f["parameters/grid"][:]

In [11]:
conf_dir = "../../configs/config.yaml"
conf = read_yaml(conf_dir)

In [12]:
prior = np.array(conf["analysis"]["grid"]["prior"]["sobol"], dtype=np.float32)

In [13]:
prior

array([[ 1.00e-01,  5.00e-01],
       [ 4.00e-01,  1.40e+00],
       [ 6.40e+01,  8.20e+01],
       [ 3.00e-02,  6.00e-02],
       [ 8.70e-01,  1.07e+00],
       [-2.00e+00, -3.33e-01],
       [-3.00e+00,  3.00e+00]], dtype=float32)

In [14]:
cosmo_grid_priors.shape

(6, 2)